In [109]:
from flask import Flask, jsonify, request
import os
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler, minmax_scale, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np
import subprocess

In [92]:
sal= pd.read_csv('data/ds_salaries.csv')

In [93]:
sal.drop(['Unnamed: 0'], axis=1, inplace=True)

In [94]:
for col in sal.columns:
    print(f"col:{col}", sal[col].unique())
    print(f"col:{col}", sal[col].nunique())
    print(f"col:{col}", sal[col].isnull().sum())
    
    

col:work_year [2020 2021 2022]
col:work_year 3
col:work_year 0
col:experience_level ['MI' 'SE' 'EN' 'EX']
col:experience_level 4
col:experience_level 0
col:employment_type ['FT' 'CT' 'PT' 'FL']
col:employment_type 4
col:employment_type 0
col:job_title ['Data Scientist' 'Machine Learning Scientist' 'Big Data Engineer'
 'Product Data Analyst' 'Machine Learning Engineer' 'Data Analyst'
 'Lead Data Scientist' 'Business Data Analyst' 'Lead Data Engineer'
 'Lead Data Analyst' 'Data Engineer' 'Data Science Consultant'
 'BI Data Analyst' 'Director of Data Science' 'Research Scientist'
 'Machine Learning Manager' 'Data Engineering Manager'
 'Machine Learning Infrastructure Engineer' 'ML Engineer' 'AI Scientist'
 'Computer Vision Engineer' 'Principal Data Scientist'
 'Data Science Manager' 'Head of Data' '3D Computer Vision Researcher'
 'Data Analytics Engineer' 'Applied Data Scientist'
 'Marketing Data Analyst' 'Cloud Data Engineer' 'Financial Data Analyst'
 'Computer Vision Software Engineer' 

In [95]:
sal.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [111]:
sal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           607 non-null    int64 
 1   experience_level    607 non-null    int64 
 2   employment_type     607 non-null    int64 
 3   job_title           607 non-null    object
 4   salary              607 non-null    int64 
 5   salary_currency     607 non-null    object
 6   salary_in_usd       607 non-null    int64 
 7   employee_residence  607 non-null    object
 8   remote_ratio        607 non-null    int64 
 9   company_location    607 non-null    object
 10  company_size        607 non-null    int64 
dtypes: int64(7), object(4)
memory usage: 52.3+ KB


In [97]:
sal.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [98]:
sal.describe().T

,count,mean,std,min,25%,50%,75%,max
work_year,607.0,2021.405272,6.921330e-01,2020.0,2021.0,2022.0,2022.0,2022.0
salary,607.0,324000.062603,1.544357e+06,4000.0,70000.0,115000.0,165000.0,30400000.0
salary_in_usd,607.0,112297.869852,7.095726e+04,2859.0,62726.0,101570.0,150000.0,600000.0
remote_ratio,607.0,70.922570,4.070913e+01,0.0,50.0,100.0,100.0,100.0


In [9]:
X_train, X_test, Y_train_y, Y_test_y = train_test_split(sal.drop('salary', axis=1), sal['salary'], test_size=0.2, random_state=42)


In [99]:
sal["work_year"] = sal["work_year"].map({2020: 0, 2021: 1, 2022: 2})
sal["experience_level"]= sal["experience_level"].map({'MI': 0, 'SE': 1, 'EN' : 2, 'EX':3})
sal["company_size"] = sal["company_size"].map({'S': 0, 'M': 1, 'L': 2})
sal["employment_type"]= sal["employment_type"].map({"FT": 0, "CT": 1, 'PT':2 , 'FL': 3})
sal["remote_ratio"] = sal["remote_ratio"].apply(lambda x: 1 if x == 50 else 2)

In [123]:
sal

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,-2.032024,-1.088876,-0.169238,0.095406,-0.164605,-1.378378,-0.457904,-1.446366,0.441454,-1.491261,1.240346
1,-2.032024,0.148854,-0.169238,1.899208,-0.041475,0.679021,2.083282,-0.460527,0.441454,-0.429997,-1.820182
2,-2.032024,0.148854,-0.169238,-1.328648,-0.154885,-1.149778,-0.046177,-1.117753,-2.265240,-1.116697,-0.289918
3,-2.032024,-1.088876,-0.169238,2.468830,-0.197008,0.679021,-1.301826,-0.953447,0.441454,-0.991843,-1.820182
4,-2.032024,0.148854,-0.169238,1.614397,-0.112761,0.679021,0.531774,0.799156,-2.265240,0.756121,1.240346
...,...,...,...,...,...,...,...,...,...,...,...
602,0.859977,0.148854,-0.169238,-0.379279,-0.110169,0.679021,0.588192,0.799156,0.441454,0.756121,-0.289918
603,0.859977,0.148854,-0.169238,-0.379279,-0.128314,0.679021,0.193263,0.799156,0.441454,0.756121,-0.289918
604,0.859977,0.148854,-0.169238,-0.853963,-0.126370,0.679021,0.235577,0.799156,0.441454,0.756121,-0.289918
605,0.859977,0.148854,-0.169238,-0.853963,-0.112761,0.679021,0.531774,0.799156,0.441454,0.756121,-0.289918


In [112]:
cl_cat= sal.select_dtypes(include='object').columns
for column in cl_cat:
    label = LabelEncoder()
    sal[column] = label.fit_transform(sal[column])


In [117]:
for col in sal.columns:
    Stand= StandardScaler()
    sal[col]= Stand.fit_transform(sal[col].values.reshape(-1,1))#usamos reshape pq fit_transfor estpara una matriz no una serie panda 

In [119]:
sal.to_csv('data/ds_salaries_clean_scal.csv', index=False)

In [129]:
X_train, X_test, y_train, y_test = train_test_split(sal.drop('salary', axis=1), sal['salary'], test_size=0.2, random_state=42)

In [131]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(485, 10) (122, 10) (485,) (122,)


In [130]:
y_train

9     -0.128962
227   -0.161365
591   -0.116096
516   -0.111141
132   -0.185084
         ...   
71    -0.185991
106   -0.057677
270   -0.162985
435   -0.164605
102    6.918609
Name: salary, Length: 485, dtype: float64

In [133]:
#guardamos el modelo en pickle
lasso = Lasso(alpha=0.1)#alpha es el parametro de regularizacion
lasso.fit(X_train, Y_train_y)#entrenamos el modelo
pickle.dump(lasso, open('data/model_salary.pkl', 'wb'))#guardamos el modelo en un archivo pickle


In [134]:
y_train.dtype

dtype('float64')